In [21]:
from sage.all import *
import random
import time

def transform_point(x_i):
    coord_sum = sum(x_i)
    x_prime_i = x_i + [-0.5 * coord_sum]  # Append the additional dimension
    x_prime_i_matrix = Matrix(RR, [x_prime_i]).transpose()
    return x_prime_i_matrix

def get_points_from_arff(file_path):
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            if not line.startswith('@') and line.strip():
                point_coords = list(map(float, line.strip().split(',')))
                points.append(point_coords)
    return points

def generate_X_i_L(x_prime_i, d):
    minus_ones = Matrix(RR, [[-1]] * (d + 1))
    X_i_L = x_prime_i.stack(minus_ones)
    return X_i_L

def generate_X_i_R(x_prime_i, d):
    ones = Matrix(RR, [[1]] * (d + 1))
    X_i_R = ones.stack(x_prime_i)
    return X_i_R

def generate_X_hat_i_L(X_i_L, alpha_hat_i_L, z_hat_x_i_L, beta_hat_i_L):
    alpha_X_i_L = alpha_hat_i_L * X_i_L
    z_hat_x_i_L_row = Matrix(RR, [[z_hat_x_i_L]] * 2)
    beta_hat_i_L_row = Matrix(RR, [[beta_hat_i_L]])
    one_row = Matrix(RR, [[1]])
    X_hat_i_L = alpha_X_i_L.stack(z_hat_x_i_L_row).stack(beta_hat_i_L_row).stack(one_row)
    return X_hat_i_L

def generate_X_hat_i_R(X_i_R, alpha_hat_i_R, z_hat_x_i_R, beta_hat_i_R):
    alpha_X_i_R = alpha_hat_i_R * X_i_R
    z_hat_x_i_R_row = Matrix(RR, [[z_hat_x_i_R]] * 2)
    beta_hat_i_R_row = Matrix(RR, [[beta_hat_i_R]])
    one_row = Matrix(RR, [[1]])
    X_hat_i_R = alpha_X_i_R.stack(z_hat_x_i_R_row).stack(beta_hat_i_R_row).stack(one_row)
    return X_hat_i_R

def generate_X_tilde_i_L(X_i_L, alpha_tilde_i_L, z_tilde_x_i_L, beta_tilde_i_L):
    alpha_X_i_L = alpha_tilde_i_L * X_i_L
    z_tilde_x_i_L_row = Matrix(RR, [[z_tilde_x_i_L]] * 2)
    beta_tilde_i_L_row = Matrix(RR, [[beta_tilde_i_L]])
    one_row = Matrix(RR, [[1]])
    X_tilde_i_L = alpha_X_i_L.stack(z_tilde_x_i_L_row).stack(beta_tilde_i_L_row).stack(one_row)
    return X_tilde_i_L

def generate_X_tilde_i_R(X_i_R, alpha_tilde_i_R, z_tilde_x_i_R, beta_tilde_i_R):
    alpha_X_i_R = alpha_tilde_i_R * X_i_R
    z_tilde_x_i_R_row = Matrix(RR, [[z_tilde_x_i_R]] * 2)
    beta_tilde_i_R_row = Matrix(RR, [[beta_tilde_i_R]])
    one_row = Matrix(RR, [[1]])
    X_tilde_i_R = alpha_X_i_R.stack(z_tilde_x_i_R_row).stack(beta_tilde_i_R_row).stack(one_row)
    return X_tilde_i_R

def random_split_matrix(matrix):
    random_matrix = Matrix(RR, matrix.nrows(), matrix.ncols(), lambda i, j: random.uniform(-2**512, 2**512) * matrix[i, j])
    return random_matrix, matrix - random_matrix

def generate_invertible_matrix(n):
    min_value = -2**512
    max_value = 2**512
    while True:
        M = Matrix(RR, n, n, lambda i, j: random.uniform(min_value, max_value))
        if M.is_invertible():
            return M

def generate_random_parameters():
    def generate_alpha_beta_z():
        min_value = 0
        max_value = 2**512
        while True:
            alpha = random.uniform(min_value, max_value)
            beta = random.uniform(min_value, alpha)
            z = random.uniform(min_value, max_value)
            if alpha > beta:  # Ensure alpha > beta
                return alpha, beta, z

    alpha_hat_i_L, beta_hat_i_L, z_hat_x_i_L = generate_alpha_beta_z()
    alpha_hat_i_R, beta_hat_i_R, z_hat_x_i_R = generate_alpha_beta_z()
    alpha_tilde_i_L, beta_tilde_i_L, z_tilde_x_i_L = generate_alpha_beta_z()
    alpha_tilde_i_R, beta_tilde_i_R, z_tilde_x_i_R = generate_alpha_beta_z()

    return (alpha_hat_i_L, z_hat_x_i_L, beta_hat_i_L,
            alpha_hat_i_R, z_hat_x_i_R, beta_hat_i_R,
            alpha_tilde_i_L, z_tilde_x_i_L, beta_tilde_i_L,
            alpha_tilde_i_R, z_tilde_x_i_R, beta_tilde_i_R)

file_path = "EEGEyeState.arff"
points = get_points_from_arff(file_path)

if len(points) < 2:
    raise ValueError("The dataset must contain at least 2 rows.")

# Randomly select two different rows
selected_indices = random.sample(range(len(points)), 2)
x_1 = points[selected_indices[0]]
x_2 = points[selected_indices[1]]

d = len(x_1)  # Subtract 1 to exclude the class label

(alpha_hat_i_L, z_hat_x_i_L, beta_hat_i_L,
 alpha_hat_i_R, z_hat_x_i_R, beta_hat_i_R,
 alpha_tilde_i_L, z_tilde_x_i_L, beta_tilde_i_L,
 alpha_tilde_i_R, z_tilde_x_i_R, beta_tilde_i_R) = generate_random_parameters()

transformed_points = [transform_point(x_i) for x_i in [x_1, x_2]]

# Placeholder dictionary for M_hat_L_u_v_w
M_hat_L_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2 * (d + 3)) for u in [1, 2] for v in [1, 2] for w in [1, 2]}

M_hat_R_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2 * (d + 3)) for u in [1, 2] for v in [1, 2] for w in [1, 2]}

M_tilde_L_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2 * (d + 3)) for u in [1, 2] for v in [1, 2] for w in [1, 2]}

M_tilde_R_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2 * (d + 3)) for u in [1, 2] for v in [1, 2] for w in [1, 2]}

CT_hat_i_L_results = {}
CT_hat_i_R_results = {}
CT_tilde_i_L_results = {}
CT_tilde_i_R_results = {}

# Start the timer
start_time = time.time()

# Print the original and transformed coordinates
print(f"Selected coordinates x_1: {x_1}")
print(f"Selected coordinates x_2: {x_2}")

for i, (x_i, x_prime_i) in enumerate(zip([x_1, x_2], transformed_points)):
    print(f"\nTransformed coordinates x_prime_{i+1}: {x_prime_i}")
    
    X_i_L = generate_X_i_L(x_prime_i, d)
    X_i_R = generate_X_i_R(x_prime_i, d)
    
    X_hat_i_L = generate_X_hat_i_L(X_i_L, alpha_hat_i_L, z_hat_x_i_L, beta_hat_i_L)
    X_hat_i_R = generate_X_hat_i_R(X_i_R, alpha_hat_i_R, z_hat_x_i_R, beta_hat_i_R)
    
    X_hat_i_L_1, X_hat_i_L_2 = random_split_matrix(X_hat_i_L)
    X_hat_i_R_1, X_hat_i_R_2 = random_split_matrix(X_hat_i_R)
    
    X_tilde_i_L = generate_X_tilde_i_L(X_i_L, alpha_tilde_i_L, z_tilde_x_i_L, beta_tilde_i_L)
    X_tilde_i_R = generate_X_tilde_i_R(X_i_R, alpha_tilde_i_R, z_tilde_x_i_R, beta_tilde_i_R)
    
    X_tilde_i_L_1, X_tilde_i_L_2 = random_split_matrix(X_tilde_i_L)
    X_tilde_i_R_1, X_tilde_i_R_2 = random_split_matrix(X_tilde_i_R)
    
    print(f"\nX_{i+1}_L =\n{X_i_L}")
    print(f"X_{i+1}_R =\n{X_i_R}")
    print(f"X_hat_{i+1}_L =\n{X_hat_i_L}")
    print(f"X_hat_{i+1}_R =\n{X_hat_i_R}")
    print(f"X_hat_{i+1}_L_1 =\n{X_hat_i_L_1}")
    print(f"X_hat_{i+1}_L_2 =\n{X_hat_i_L_2}")
    print(f"X_hat_{i+1}_R_1 =\n{X_hat_i_R_1}")
    print(f"X_hat_{i+1}_R_2 =\n{X_hat_i_R_2}")
    
    print(f"X_tilde_{i+1}_L =\n{X_tilde_i_L}")
    print(f"X_tilde_{i+1}_R =\n{X_tilde_i_R}")
    print(f"X_tilde_{i+1}_L_1 =\n{X_tilde_i_L_1}")
    print(f"X_tilde_{i+1}_L_2 =\n{X_tilde_i_L_2}")
    print(f"X_tilde_{i+1}_R_1 =\n{X_tilde_i_R_1}")
    print(f"X_tilde_{i+1}_R_2 =\n{X_tilde_i_R_2}")

    for u in [1, 2]:
        for v in [1, 2]:
            for w in [1, 2]:
                if u == 1:
                    X_hat_i_L_u = X_hat_i_L_1
                elif u == 2:
                    X_hat_i_L_u = X_hat_i_L_2
                M_hat_L_u_v_w = M_hat_L_u_v_w_dict[(u, v, w)]
                CT_hat_i_L_u_v_w = X_hat_i_L_u.transpose() * M_hat_L_u_v_w
                CT_hat_i_L_results[(i + 1, u, v, w)] = CT_hat_i_L_u_v_w
                
    for u in [1, 2]:
        for v in [1, 2]:
            for w in [1, 2]:
                if u == 1:
                    X_hat_i_R_u = X_hat_i_R_1
                elif u == 2:
                    X_hat_i_R_u = X_hat_i_R_2
                M_hat_R_u_v_w = M_hat_R_u_v_w_dict[(u, v, w)]
                CT_hat_i_R_u_v_w = M_hat_R_u_v_w.inverse() * X_hat_i_R_u
                CT_hat_i_R_results[(i + 1, u, v, w)] = CT_hat_i_R_u_v_w
                
    for u in [1, 2]:
        for v in [1, 2]:
            for w in [1, 2]:
                if u == 1:
                    X_tilde_i_L_u = X_tilde_i_L_1
                elif u == 2:
                    X_tilde_i_L_u = X_tilde_i_L_2
                M_tilde_L_u_v_w = M_tilde_L_u_v_w_dict[(u, v, w)]
                CT_tilde_i_L_u_v_w = X_tilde_i_L_u.transpose() * M_tilde_L_u_v_w 
                CT_tilde_i_L_results[(i + 1, u, v, w)] = CT_tilde_i_L_u_v_w
                
    for u in [1, 2]:
        for v in [1, 2]:
            for w in [1, 2]:
                if u == 1:
                    X_tilde_i_R_u = X_tilde_i_R_1
                elif u == 2:
                    X_tilde_i_R_u = X_tilde_i_R_2
                M_tilde_R_u_v_w = M_tilde_R_u_v_w_dict[(u, v, w)]
                CT_tilde_i_R_u_v_w = M_tilde_R_u_v_w.inverse() * X_tilde_i_R_u
                CT_tilde_i_R_results[(i + 1, u, v, w)] = CT_tilde_i_R_u_v_w

for key, value in CT_hat_i_L_results.items():
    i, u, v, w = key
    print(f"CT_hat_{i}_L_{u}_{v}_{w} =\n{value}")   
        
for key, value in CT_hat_i_R_results.items():
    i, u, v, w = key
    print(f"CT_hat_{i}_R_{u}_{v}_{w} =\n{value}")
    
for key, value in CT_tilde_i_L_results.items():
    i, u, v, w = key
    print(f"CT_tilde_{i}_L_{u}_{v}_{w} =\n{value}")
    
for key, value in CT_tilde_i_R_results.items():
    i, u, v, w = key
    print(f"CT_tilde_{i}_R_{u}_{v}_{w} =\n{value}")

# Stop the timer
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


Selected coordinates x_1: [4242.56, 4020.0, 4249.74, 4132.82, 4341.03, 4611.28, 4048.72, 4595.9, 4182.56, 4181.54, 4130.77, 4244.62, 4481.54, 4279.49, 0.0]
Selected coordinates x_2: [4300.0, 3971.79, 4276.92, 4120.0, 4355.38, 4634.87, 4083.59, 4644.1, 4215.9, 4255.9, 4226.15, 4291.28, 4650.77, 4392.82, 1.0]

Transformed coordinates x_prime_1: [ 4242.56000000000]
[ 4020.00000000000]
[ 4249.74000000000]
[ 4132.82000000000]
[ 4341.03000000000]
[ 4611.28000000000]
[ 4048.72000000000]
[ 4595.90000000000]
[ 4182.56000000000]
[ 4181.54000000000]
[ 4130.77000000000]
[ 4244.62000000000]
[ 4481.54000000000]
[ 4279.49000000000]
[0.000000000000000]
[-29871.2850000000]

X_1_L =
[ 4242.56000000000]
[ 4020.00000000000]
[ 4249.74000000000]
[ 4132.82000000000]
[ 4341.03000000000]
[ 4611.28000000000]
[ 4048.72000000000]
[ 4595.90000000000]
[ 4182.56000000000]
[ 4181.54000000000]
[ 4130.77000000000]
[ 4244.62000000000]
[ 4481.54000000000]
[ 4279.49000000000]
[0.000000000000000]
[-29871.2850000000]
[-1.00

In [20]:
from sage.all import *
import random
import time

def transform_point_q(points):
    q_j_list = points + [1]
    q_j_matrix = column_matrix(RR, q_j_list)
    return q_j_matrix

def read_random_point_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        data_lines = [line for line in lines if not line.startswith('@')]
        random_line = random.choice(data_lines).strip()
        point_coords = list(map(float, random_line.split(',')))  # Split by comma
        return point_coords

def generate_Q_j(q_prime_j):
    n = q_prime_j.nrows()
    diag_q_prime_j = diagonal_matrix(RR, q_prime_j.list())
    zero_matrix = matrix(RR, n, n)
    Q_j = block_matrix(2, 2, [diag_q_prime_j, zero_matrix, zero_matrix, diag_q_prime_j])
    return Q_j

def generate_invertible_matrix(n):
    while True:
        random_entries = [[random.uniform(-2**512, 2**512) for _ in range(n)] for _ in range(n)]
        M = Matrix(RR, random_entries)
        if M.is_invertible():
            return M

# Start the timer
start_time = time.time()

# Adjust the value of d to 15
d = 15

# Read a random point from the file
filename = 'EEGEyeState.arff'
point_coords = read_random_point_from_file(filename)
print(f"Selected random point coordinates: {point_coords}")

# Transform the point
transformed_points = [transform_point_q(point_coords)]

# Generate Q_1 and Q_j
Q_1 = generate_Q_j(transformed_points[0])
Q_j = generate_Q_j(transformed_points[0])  # Assuming Q_j is based on the same point for simplicity

print(f"Q_1 =\n{Q_1}")
print(f"Q_j =\n{Q_j}")

# Generate random invertible matrices
M_hat_L_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2*d+6) for u in [1, 2] for v in [1, 2] for w in [1, 2]}
M_hat_R_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2*d+6) for u in [1, 2] for v in [1, 2] for w in [1, 2]}
M_tilde_L_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2*d+6) for u in [1, 2] for v in [1, 2] for w in [1, 2]}
M_tilde_R_u_v_w_dict = {(u, v, w): generate_invertible_matrix(2*d+6) for u in [1, 2] for v in [1, 2] for w in [1, 2]}

# Generate alpha and alpha_matrix
alpha = random.uniform(0, 2**512)
alpha_matrix = alpha * Q_j

print(f"alpha = {alpha}")
print(f"alpha * Q_j =\n{alpha_matrix}")

# Generate zero column matrix of dimension equal to alpha_matrix
zero_column_matrix = column_matrix(RR, [0] * alpha_matrix.nrows())

print(f"Zero column matrix =\n{zero_column_matrix}")

range_min, range_max = 0, 2**512
beta = random.uniform(range_min, alpha)  # Ensuring alpha > beta
z = random.uniform(range_min, range_max)
gamma = random.uniform(range_min, range_max)

# Construct Q_hat_j matrix
dimension_alpha_matrix = alpha_matrix.dimensions()[0]
Q_hat_j = block_matrix(RR, [
    [alpha_matrix, zero_column_matrix, zero_column_matrix, zero_column_matrix, zero_column_matrix],
    [zero_column_matrix.transpose(), z, 0, 0, 0],
    [zero_column_matrix.transpose(), 0, z, 0, 0],
    [zero_column_matrix.transpose(), 0, 0, beta, 0],
    [zero_column_matrix.transpose(), 0, 0, 0, gamma]
])

print(f"Q_hat_j =\n{Q_hat_j}")

# Construct Q_tilde_j matrix
Q_tilde_j = block_matrix(RR, [
    [alpha_matrix, zero_column_matrix, zero_column_matrix, zero_column_matrix, zero_column_matrix],
    [zero_column_matrix.transpose(), z, 0, 0, 0],
    [zero_column_matrix.transpose(), 0, z, 0, 0],
    [zero_column_matrix.transpose(), 0, 0, beta, 0],
    [zero_column_matrix.transpose(), 0, 0, 0, -gamma]
])

print(f"Q_tilde_j =\n{Q_tilde_j}")

# Randomly split Q_hat_j into Q_hat_j_1 and Q_hat_j_2
rows, cols = Q_hat_j.dimensions()
Q_hat_j_1_entries = [[random.uniform(-2**512, 2**512) for _ in range(cols)] for _ in range(rows)]
Q_hat_j_1 = Matrix(RR, Q_hat_j_1_entries)

Q_hat_j_2 = Q_hat_j - Q_hat_j_1

print(f"Q_hat_j_1 =\n{Q_hat_j_1}")
print(f"Q_hat_j_2 =\n{Q_hat_j_2}")

# Randomly split Q_tilde_j into Q_tilde_j_1 and Q_tilde_j_2
rows, cols = Q_tilde_j.dimensions()
Q_tilde_j_1_entries = [[random.uniform(-2**512, 2**512) for _ in range(cols)] for _ in range(rows)]
Q_tilde_j_1 = Matrix(RR, Q_tilde_j_1_entries)

Q_tilde_j_2 = Q_tilde_j - Q_tilde_j_1

print(f"Q_tilde_j_1 =\n{Q_tilde_j_1}")
print(f"Q_tilde_j_2 =\n{Q_tilde_j_2}")

# Computing the ciphertexts for each combination of u, v, w in {1, 2}
CT_hat_j_results = {}
CT_tilde_j_results = {}

for j, q_prime_j in enumerate(transformed_points, start=1):
    for u in [1, 2]:
        for v in [1, 2]:
            for w in [1, 2]:
                if v == 1:
                    Q_hat_j_v = Q_hat_j_1
                    Q_tilde_j_v = Q_tilde_j_1
                elif v == 2:
                    Q_hat_j_v = Q_hat_j_2
                    Q_tilde_j_v = Q_tilde_j_2
                
                M_hat_L_u_v_w = M_hat_L_u_v_w_dict[(u, v, w)]
                M_tilde_L_u_v_w = M_tilde_L_u_v_w_dict[(u, v, w)]
                
                CT_hat_j_u_v_w = M_hat_L_u_v_w.inverse() * Q_hat_j_v * M_hat_L_u_v_w
                CT_tilde_j_u_v_w = M_tilde_L_u_v_w.inverse() * Q_tilde_j_v * M_tilde_L_u_v_w
                
                CT_hat_j_results[(1, u, v, w)] = CT_hat_j_u_v_w
                CT_tilde_j_results[(j, u, v, w)] = CT_tilde_j_u_v_w

for key, value in CT_hat_j_results.items():
    print(f"CT_hat_{key} =\n{value}")

for key, value in CT_tilde_j_results.items():
    print(f"CT_tilde_{key} =\n{value}")

# Stop the timer
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


Selected random point coordinates: [4288.72, 4038.46, 4262.56, 4120.0, 4347.69, 4633.85, 4062.05, 4608.21, 4198.46, 4231.79, 4195.9, 4264.62, 4595.38, 4339.49, 0.0]
Q_1 =
[ 4288.72000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000|0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000]
[0.000000000000000  4038.46000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.000000000000000 0.00000000000000

In [23]:
#Evaluation by cloud
import time

# Initialize the result for z_hat_1_j_2
z_hat_1_j_2 = 0
z_tilde_1_j_2 = 0

# Start the timer
start_time = time.time()

# Loop through u, v, w to calculate the summation
for u in [1, 2]:
    for v in [1, 2]:
        for w in [1, 2]:
            CT_hat_1_L_u_v_w = CT_hat_i_L_results[(1, u, v, w)]
            CT_hat_j_u_v_w = CT_hat_j_results[(j, u, v, w)]
            CT_hat_2_R_u_v_w = CT_hat_i_R_results[(2, u, v, w)]
            
            # Add the product to z_hat_1_j_2
            z_hat_1_j_2 += CT_hat_1_L_u_v_w * CT_hat_j_u_v_w * CT_hat_2_R_u_v_w

print(f"z_hat_1_j_2 =\n{z_hat_1_j_2}")

# Loop through u, v, w to calculate the summation
for u in [1, 2]:
    for v in [1, 2]:
        for w in [1, 2]:
            CT_tilde_1_L_u_v_w = CT_tilde_i_L_results[(1, u, v, w)]
            CT_tilde_j_u_v_w = CT_tilde_j_results[(j, u, v, w)]
            CT_tilde_2_R_u_v_w = CT_tilde_i_R_results[(2, u, v, w)]
            
            # Add the product to z_hat_1_j_2
            z_tilde_1_j_2 += CT_tilde_1_L_u_v_w * CT_tilde_j_u_v_w * CT_tilde_2_R_u_v_w

print(f"z_tilde_1_j_2 =\n{z_tilde_1_j_2}")

z_i_j_2 = z_hat_1_j_2 + z_tilde_1_j_2
print("z_i_j_2=")
print(z_i_j_2)

# Stop the timer
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

z_hat_1_j_2 =
[-2.12747682714706e783]
z_tilde_1_j_2 =
[2.99995660487305e783]
z_i_j_2=
[8.72479777725994e782]
Execution time: 0.022258996963500977 seconds


In [27]:
# Importing necessary libraries
from sage.all import *

# Function to calculate Euclidean distance for 15-dimensional points
def euclidean_distance(coord1, coord2, coord3):
    if len(coord1.split(',')) != 15 or len(coord2.split(',')) != 15 or len(coord3.split(',')) != 15:
        raise ValueError("Each coordinate should contain exactly 15 entries.")
    
    # Convert input strings to lists of floats
    coord1 = list(map(float, coord1.split(',')))
    coord2 = list(map(float, coord2.split(',')))
    coord3 = list(map(float, coord3.split(',')))
    
    # Calculating Euclidean distances
    distance1 = sqrt(sum((coord3[i] - coord1[i])**2 for i in range(15)))
    distance2 = sqrt(sum((coord3[i] - coord2[i])**2 for i in range(15)))
    
    return distance1, distance2

# Getting input from the user
try:
    coord1 = input("Enter coordinates of the first 15-dimensional point (separated by commas): ")
    coord2 = input("Enter coordinates of the second 15-dimensional point (separated by commas): ")
    coord3 = input("Enter coordinates of the third 15-dimensional point (separated by commas): ")

    # Calculating Euclidean distances
    distance1, distance2 = euclidean_distance(coord1, coord2, coord3)

    # Displaying the results
    print(f"The Euclidean distance between the first and third 15-dimensional points is {distance1:.4f}")
    print(f"The Euclidean distance between the second and third 15-dimensional points is {distance2:.4f}")
    
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Enter coordinates of the first 15-dimensional point (separated by commas): 4242.56, 4020.0, 4249.74, 4132.82, 4341.03, 4611.28, 4048.72, 4595.9, 4182.56, 4181.54, 4130.77, 4244.62, 4481.54, 4279.49, 0.0
Enter coordinates of the second 15-dimensional point (separated by commas): 4300.0, 3971.79, 4276.92, 4120.0, 4355.38, 4634.87, 4083.59, 4644.1, 4215.9, 4255.9, 4226.15, 4291.28, 4650.77, 4392.82, 1.0
Enter coordinates of the third 15-dimensional point (separated by commas): 4288.72, 4038.46, 4262.56, 4120.0, 4347.69, 4633.85, 4062.05, 4608.21, 4198.46, 4231.79, 4195.9, 4264.62, 4595.38, 4339.49, 0.0
The Euclidean distance between the first and third 15-dimensional points is 166.3206
The Euclidean distance between the second and third 15-dimensional points is 122.5356


In [30]:
if (z_i_j_2 > 0 and distance1 < distance2) or (z_i_j_2 < 0 and distance1 >= distance2):
    print("Algorithm is correct.")
else:
    print("Algorithm is incorrect.")

Algorithm is incorrect.


In [34]:
#Code for clodu to compute the kNN.
'''
import heapq
import time
from sage.all import *

# Function to read ARFF file and parse the data
def load_arff_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('@') and line.strip():
                data.append(list(map(float, line.strip().split(','))))
    return data

# Function to calculate Euclidean distance
def euclidean_distance(row1, row2):
    return sqrt(sum((row1[i] - row2[i])**2 for i in range(len(row1))))

# Function to get top k elements from a max heap
def get_top_k_elements(max_heap, k):
    return heapq.nlargest(k, max_heap)

# Main function
def main(file_path, k):
    # Start the timer
    start_time = time.time()

    # Load data
    data = load_arff_file(file_path)
    
    # Prompt the user to input a row of data separated by commas
    user_input = input("Enter a 15-dimensional row (separated by commas): ")
    selected_row = list(map(float, user_input.split(',')))
    
    # Ensure the input row has exactly 15 entries
    if len(selected_row) != 15:
        print("Error: The input row must contain exactly 15 entries.")
        return

    # Compute Euclidean distances and store them in a max heap
    max_heap = []
    for i in range(len(data)):
        distance = euclidean_distance(selected_row, data[i])
        heapq.heappush(max_heap, (-distance, i))  # Use negative distance for max heap

    # Get the top k nodes
    top_k_elements = get_top_k_elements(max_heap, k)

    # Stop the timer
    end_time = time.time()
    execution_time = end_time - start_time

    # Display the results
    print(f"Top {k} nodes (distance, index):")
    for dist, idx in top_k_elements:
        print(f"Distance: {-dist:.4f}, Index: {idx}")

    print(f"Execution time: {execution_time:.4f} seconds")

# File path to the ARFF file
file_path = 'EEGEyeState.arff'
# Value of k
k = int(input("Enter the value of k: "))

# Run the main function
main(file_path, k)

'''

Enter the value of k: 4
Enter a 15-dimensional row (separated by commas): 4288.72, 4038.46, 4262.56, 4120.0, 4347.69, 4633.85, 4062.05, 4608.21, 4198.46, 4231.79, 4195.9, 4264.62, 4595.38, 4339.49, 0.0
Top 4 nodes (distance, index):
Distance: 0.0000, Index: 13762
Distance: 11.0056, Index: 13763
Distance: 17.8442, Index: 13880
Distance: 20.2748, Index: 13738
Execution time: 12.8434 seconds


In [37]:
import heapq
import time
from sage.all import *

# Function to read ARFF file and parse the data
def load_arff_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('@') and line.strip():
                data.append(list(map(float, line.strip().split(','))))
    return data

# Function to calculate Euclidean distance
def euclidean_distance(row1, row2):
    return sqrt(sum((row1[i] - row2[i])**2 for i in range(len(row1))))

# Function to get top k elements from a max heap
def get_top_k_elements(max_heap, k):
    return heapq.nlargest(k, max_heap)

# Function to validate algorithm correctness
def validate_algorithm(selected_row, top_k_elements):
    distance1 = -top_k_elements[0][0]  # Distance of the top element in the max heap
    distance2 = -top_k_elements[1][0]  # Distance of the second top element in the max heap
    z_i_j_2 = selected_row[14]  # Assuming 15-dimensional row, and z_i_j_2 corresponds to the 15th element

    if (z_i_j_2 > 0 and distance1 < distance2) or (z_i_j_2 < 0 and distance1 >= distance2):
        return "correct"
    else:
        return "incorrect"

# Main function
def main(file_path, k):
    # Start the timer
    start_time = time.time()

    # Load data
    data = load_arff_file(file_path)
    
    # Prompt the user to input a row of data separated by commas
    user_input = input("Enter a 15-dimensional row (separated by commas): ")
    selected_row = list(map(float, user_input.split(',')))
    
    # Ensure the input row has exactly 15 entries
    if len(selected_row) != 15:
        print("Error: The input row must contain exactly 15 entries.")
        return

    # Compute Euclidean distances and store them in a max heap
    max_heap = []
    for i in range(len(data)):
        distance = euclidean_distance(selected_row, data[i])
        heapq.heappush(max_heap, (-distance, i))  # Use negative distance for max heap

    # Get the top k nodes
    top_k_elements = get_top_k_elements(max_heap, k)

    # Validate algorithm correctness
    correctness = validate_algorithm(selected_row, top_k_elements)

    # Stop the timer
    end_time = time.time()
    execution_time = end_time - start_time

    # Display the results
    if correctness == "correct":
        print(f"Top {k} nodes (distance, index):")
        for dist, idx in top_k_elements:
            print(f"Distance: {-dist:.4f}, Index: {idx}")

        print(f"Execution time: {execution_time:.4f} seconds")
    else:
        print(f"Unable to compute the value of k: {k}")

# File path to the ARFF file
file_path = 'EEGEyeState.arff'
# Value of k
k = int(input("Enter the value of k: "))

# Run the main function
main(file_path, k)


Enter the value of k: 2
Enter a 15-dimensional row (separated by commas): 4288.72, 4038.46, 4262.56, 4120.0, 4347.69, 4633.85, 4062.05, 4608.21, 4198.46, 4231.79, 4195.9, 4264.62, 4595.38, 4339.49, 0.0
Unable to compute the value of k: 2
